In [1]:
import json
import types
from collections import OrderedDict
from pathlib import Path
from pprint import pprint

import torch
import torch.nn as nn

from dataset import get_new_dataloader
from funcs import get_dataloaders
from models import PreActResNet18, WideResNet
from train import epoch, epoch_test


In [2]:
# constant
CWD_PATH = Path.cwd()
MODEL_DIR_PATH = CWD_PATH.parent / "models/CIFAR10-CINIC10-EXCL"
LABELS = [
    "model_teacher_normalized_0.0",
    "model_teacher_normalized_0.1",
    "model_teacher_normalized_0.2",
    "model_teacher_normalized_0.3",
    "model_teacher_normalized_0.5",
    "model_teacher_normalized_0.7",
    "model_teacher_normalized_0.8",
    "model_teacher_normalized_0.9",
    "model_teacher_normalized_1.0",
]

In [3]:
def load_model(
    model_type: str,
    location: str,
    model_normalize: bool,
):
    device = torch.device("cuda:{0}".format(0) if torch.cuda.is_available() else "cpu")

    # load model CIFAR10
    if model_type == "pre-act-18":
        model = PreActResNet18(num_classes=10, normalize=model_normalize)
    elif model_type == "independent":
        model = WideResNet(
            n_classes=10,
            depth=16,  # deep_full for CIFAR10
            widen_factor=1,
            normalize=model_normalize,
            dropRate=0.3,
        )
    else:
        model = WideResNet(
            n_classes=10,
            depth=28,  # deep_full for CIFAR10
            widen_factor=10,
            normalize=model_normalize,
            dropRate=0.3,
        )

    try:
        model = model.to(device)
        model.load_state_dict(torch.load(location, map_location=device))
    except:
        model = nn.DataParallel(model).to(device)
        model.load_state_dict(torch.load(location, map_location=device))
    model.eval()
    return model


In [4]:
model_name__acc = OrderedDict()

# model performance in FE
for label in LABELS:
    model_path = MODEL_DIR_PATH / label / "final.pt"
    print(model_path)
    model = load_model(
        model_type="teacher",
        location=str(model_path),
        model_normalize=True,
    )

    train_loader, test_loader = get_dataloaders(
        dataset="CIFAR10",
        batch_size=1000,
        pseudo_labels=False,
        normalize=False,
        train_shuffle=False,
    )

    args = types.SimpleNamespace()
    device = torch.device("cuda:{0}".format(0) if torch.cuda.is_available() else "cpu")
    args.device = device
    args.mode = "teacher"

    _, train_acc = epoch(
        args,
        train_loader,
        model,
        teacher=None,
        lr_schedule=None,
        epoch_i=None,
        opt=None,
    )
    _, test_acc = epoch_test(
        args,
        test_loader,
        model,
    )
    print(train_acc)
    print(test_acc)

    model_name__acc[label] = {
        "train_acc": train_acc,
        "test_acc": test_acc,
    }



/data/weijing/DI-ting_cifar-cinic/models/CIFAR10-CINIC10-EXCL/model_teacher_normalized_0.0/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:32<00:00,  1.52it/s]


0.8003
0.7983
/data/weijing/DI-ting_cifar-cinic/models/CIFAR10-CINIC10-EXCL/model_teacher_normalized_0.1/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:27<00:00,  1.79it/s]


0.89872
0.8859
/data/weijing/DI-ting_cifar-cinic/models/CIFAR10-CINIC10-EXCL/model_teacher_normalized_0.2/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


...

0.9875
0.9365
/data/weijing/DI-ting_cifar-cinic/models/CIFAR10-CINIC10-EXCL/model_teacher_normalized_0.9/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:27<00:00,  1.79it/s]


0.99446
0.9417
/data/weijing/DI-ting_cifar-cinic/models/CIFAR10-CINIC10-EXCL/model_teacher_normalized_1.0/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:28<00:00,  1.78it/s]


1.0
0.9424


In [5]:
pprint(model_name__acc)

# Serializing json
json_object = json.dumps(model_name__acc, indent=2)
 
# Writing to sample.json
with open("fe_performance.json", "w") as outfile:
    outfile.write(json_object)

OrderedDict([('model_teacher_normalized_0.0',
              {'test_acc': 0.7983, 'train_acc': 0.8003}),
             ('model_teacher_normalized_0.1',
              {'test_acc': 0.8859, 'train_acc': 0.89872}),
             ('model_teacher_normalized_0.2',
              {'test_acc': 0.9113, 'train_acc': 0.92894}),
             ('model_teacher_normalized_0.3',
              {'test_acc': 0.9164, 'train_acc': 0.94418}),
             ('model_teacher_normalized_0.5',
              {'test_acc': 0.926, 'train_acc': 0.96552}),
             ('model_teacher_normalized_0.7',
              {'test_acc': 0.9299, 'train_acc': 0.98104}),
             ('model_teacher_normalized_0.8',
              {'test_acc': 0.9365, 'train_acc': 0.9875}),
             ('model_teacher_normalized_0.9',
              {'test_acc': 0.9417, 'train_acc': 0.99446}),
             ('model_teacher_normalized_1.0',
              {'test_acc': 0.9424, 'train_acc': 1.0})])


In [6]:
import pandas as pd

model_label_col = [
    "0 CIFAR | 100 CINIC",
    "10 CIFAR | 90 CINIC",
    "20 CIFAR | 80 CINIC",
    "30 CIFAR | 70 CINIC",
    "50 CIFAR | 50 CINIC",
    "70 CIFAR | 30 CINIC",
    "80 CIFAR | 20 CINIC",
    "90 CIFAR | 10 CINIC",
    "100 CIFAR | 0 CINIC",
]
train_acc_col = []
test_acc_col = []

for model_label, acc_data in model_name__acc.items():
    train_acc_col.append(acc_data["train_acc"])
    test_acc_col.append(acc_data["test_acc"])

data = {
    "Model": model_label_col,
    "Train Acc": train_acc_col,
    "Test Acc": test_acc_col,
}

pd.DataFrame.from_dict(data)


,Model,Train Acc,Test Acc
0,0 CIFAR | 100 CINIC,0.80030,0.7983
1,10 CIFAR | 90 CINIC,0.89872,0.8859
2,20 CIFAR | 80 CINIC,0.92894,0.9113
3,30 CIFAR | 70 CINIC,0.94418,0.9164
4,50 CIFAR | 50 CINIC,0.96552,0.9260
5,70 CIFAR | 30 CINIC,0.98104,0.9299
6,80 CIFAR | 20 CINIC,0.98750,0.9365
7,90 CIFAR | 10 CINIC,0.99446,0.9417
8,100 CIFAR | 0 CINIC,1.00000,0.9424


## Read

In [9]:
with open("fe_performance.json", "r") as outfile:
    data = json.load(outfile)
    for model_label, acc_dict in data.items():
        print(f"{acc_dict['test_acc']:.3f}")

0.398
0.140
0.394
0.942
1.000
0.932
0.941
0.997
0.930
0.392
0.363
0.214
0.100
0.100
0.100
0.519
0.182
0.596
0.377
0.146
0.453
0.939
1.000
0.933


In [11]:
for label in LABELS:
    l1 = label[0]
    l2 = label[1]
    l3 = label[2]

    model_type_str = L1__MODEL_TYPE_MAP[l1]
    model_norm_str = "normalized" if L2__MODEL_NORM_MAP[l2] else "unnormalized"
    data_norm_str = "normalized" if L2__DATA_NORM_MAP[l2] else "unnormalized"
    load_norm_str = "normalized" if L3__LOAD_NORM_MAP[l3] else "unnormalized"

    file_name = f"model_{model_type_str}_model-{model_norm_str}_data-{data_norm_str}_load-model-{load_norm_str}"
    key_name = f"{model_type_str}_model-{model_norm_str}_data-{data_norm_str}_load-model-{load_norm_str}"
    display_name = f"{L1__MODEL_TYPE_DISPLAY_MAP[l1]} MODEL-{model_norm_str} DATA-{data_norm_str} LOAD-model-{load_norm_str}"
    print(display_name)
    # print(key_name)

Teacher MODEL-normalized DATA-normalized LOAD-model-normalized
Independent MODEL-normalized DATA-normalized LOAD-model-normalized
Diff. Arch. MODEL-normalized DATA-normalized LOAD-model-normalized
Teacher MODEL-normalized DATA-unnormalized LOAD-model-normalized
Independent MODEL-normalized DATA-unnormalized LOAD-model-normalized
Diff. Arch. MODEL-normalized DATA-unnormalized LOAD-model-normalized
Teacher MODEL-unnormalized DATA-normalized LOAD-model-normalized
Independent MODEL-unnormalized DATA-normalized LOAD-model-normalized
Diff. Arch. MODEL-unnormalized DATA-normalized LOAD-model-normalized
Teacher MODEL-unnormalized DATA-unnormalized LOAD-model-normalized
Independent MODEL-unnormalized DATA-unnormalized LOAD-model-normalized
Diff. Arch. MODEL-unnormalized DATA-unnormalized LOAD-model-normalized
Teacher MODEL-normalized DATA-normalized LOAD-model-unnormalized
Independent MODEL-normalized DATA-normalized LOAD-model-unnormalized
Diff. Arch. MODEL-normalized DATA-normalized LOAD-mode